In [ ]:
%pip install 'unified_planning[grpc]'

The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.

zsh:1: no matches found: unified-planning[pyperplan,tamer,plot]
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install up-enhsp

Note: you may need to restart the kernel to use updated packages.


In [1]:
from unified_planning.shortcuts import *
import unified_planning as up
from unified_planning.io import PDDLWriter

# Silenciar créditos del planner
up.shortcuts.get_environment().credits_stream = None

def crear_problema_lightsout(tamaño: int) -> Problem:
    Celda = UserType('Celda')
    problema = Problem('LightsOut')

    estado = Fluent('estado', BoolType(), c=Celda)
    vecino = Fluent('vecino', BoolType(), c1=Celda, c2=Celda)

    problema.add_fluent(estado, default_initial_value=False)
    problema.add_fluent(vecino, default_initial_value=False)

    pulsar = InstantaneousAction('pulsar', c=Celda)
    c = pulsar.parameter('c')
    pulsar.add_effect(estado(c), Not(estado(c)))
    problema.add_action(pulsar)

    celdas = [Object(f'[{i},{j}]', Celda) for i in range(tamaño) for j in range(tamaño)]
    problema.add_objects(celdas)

    for i in range(tamaño):
        for j in range(tamaño):
            c_actual = next(obj for obj in celdas if obj.name == f'[{i},{j}]')
            vecinos = []
            if i > 0: vecinos.append((i-1, j))
            if i < tamaño - 1: vecinos.append((i+1, j))
            if j > 0: vecinos.append((i, j-1))
            if j < tamaño - 1: vecinos.append((i, j+1))
            for vi, vj in vecinos:
                c_vecino = next(obj for obj in celdas if obj.name == f'[{vi},{vj}]')
                problema.set_initial_value(vecino(c_actual, c_vecino), True)

    for c1 in celdas:
        for c2 in celdas:
            cond = And(Equals(pulsar.parameter('c'), c1), vecino(c1, c2))
            pulsar.add_effect(estado(c2), Not(estado(c2)), cond)

    for c_obj in celdas:
        problema.set_initial_value(estado(c_obj), False)
        problema.add_goal(estado(c_obj))

    return problema

def resolver_lightsout(tamaño: int):
    problema = crear_problema_lightsout(tamaño)
    print(f"\nResolviendo tablero de tamaño {tamaño}x{tamaño}...\n")

    # Aquí se generan los archivos PDDL
    w = PDDLWriter(problema, rewrite_bool_assignments=True)
    w.write_domain('Dominio.pddl')
    w.write_problem('Problema.pddl')

# Aquí definimos el tamaño del tablero
resolver_lightsout(tamaño=3)



Resolviendo tablero de tamaño 3x3...



In [4]:
from collections import deque

def crear_vecinos(tamaño):
    vecinos = {}
    for i in range(tamaño):
        for j in range(tamaño):
            pos = f'[{i},{j}]'
            vecinos[pos] = []
            if i > 0:
                vecinos[pos].append(f'[{i-1},{j}]')
            if i < tamaño - 1:
                vecinos[pos].append(f'[{i+1},{j}]')
            if j > 0:
                vecinos[pos].append(f'[{i},{j-1}]')
            if j < tamaño - 1:
                vecinos[pos].append(f'[{i},{j+1}]')
    return vecinos

def aplicar_accion(estado_actual, celda_pulsada, vecinos):
    nuevo_estado = estado_actual.copy()
    # Esto invierte la celda pulsada
    nuevo_estado[celda_pulsada] = not nuevo_estado[celda_pulsada]
    # Esto invierte los vecinos
    for v in vecinos[celda_pulsada]:
        nuevo_estado[v] = not nuevo_estado[v]
    return nuevo_estado

def es_objetivo(estado_actual):
    return all(estado_actual.values())

def similitud(estado_actual):
    return sum(estado_actual.values())

def busqueda_anchura(tamaño, n=3):
    celdas = [f'[{i},{j}]' for i in range(tamaño) for j in range(tamaño)]
    vecinos = crear_vecinos(tamaño)
    estado_inicial = {c: False for c in celdas}

    cola = deque()
    cola.append( (estado_inicial, []) )
    visitados = set()

    while cola:
        estado_actual, plan = cola.popleft()
        estado_frozenset = frozenset(estado_actual.items())
        if estado_frozenset in visitados:
            continue
        visitados.add(estado_frozenset)

        if es_objetivo(estado_actual):
            return plan

        sucesores = []
        for c in celdas:
            nuevo_estado = aplicar_accion(estado_actual, c, vecinos)
            nuevo_plan = plan + [f'pulsar {c}']
            sucesores.append( (nuevo_estado, nuevo_plan) )

        # Si alguno es objetivo, devolvemos inmediatamente
        for est, p in sucesores:
            if es_objetivo(est):
                return p

        # Tomamos los n mejores sucesores según similitud
        sucesores.sort(key=lambda x: similitud(x[0]), reverse=True)
        cola.extend(sucesores[:n])

    return None


plan = busqueda_anchura(3, n=5)
if plan:
    print(f"Plan encontrado en {len(plan)} pasos:")
    for i, paso in enumerate(plan, 1):
        print(f"Paso {i}: {paso}")
else:
    print("No se encontró plan con esta búsqueda heurística.")

Plan encontrado en 5 pasos:
Paso 1: pulsar [1,1]
Paso 2: pulsar [0,0]
Paso 3: pulsar [0,2]
Paso 4: pulsar [2,0]
Paso 5: pulsar [2,2]


In [5]:
from unified_planning.io import PDDLReader
from unified_planning.shortcuts import *
from collections import deque

def get_vecinos_from_problem(problem):
    vecinos = {}
    celdas = [str(o) for o in problem.objects(typename=problem.user_types[0])]

    for c in celdas:
        vecinos[c] = []

    for fluent, val in problem.initial_values.items():
        if 'vecino' in str(fluent) and val.constant_value():
            c1 = str(fluent.args[0])
            c2 = str(fluent.args[1])
            vecinos[str(c1)].append(str(c2))
    return vecinos

def estado_inicial_from_problem(problem):
    estado = {}
    for fluent, val in problem.initial_values.items():
        if 'estado' in str(fluent):
            celda = str(fluent.args[0])
            estado[celda] = val.constant_value()
    return estado

def aplicar_accion_estado(estado, celda, vecinos):
    nuevo_estado = estado.copy()
    nuevo_estado[celda] = not nuevo_estado[celda]
    for v in vecinos[celda]:
        nuevo_estado[v] = not nuevo_estado[v]
    return nuevo_estado

def es_objetivo(estado):
    return all(estado.values())

def similitud(estado):
    return sum(estado.values())

def busqueda_heuristica_pddl(problem, n=3):
    vecinos = get_vecinos_from_problem(problem)
    estado_inicial = estado_inicial_from_problem(problem)
    celdas = list(vecinos.keys())
    cola = deque()
    cola.append((estado_inicial, []))
    visitados = set()
    print(vecinos)
    print(estado_inicial)
    while cola:
        estado_actual, plan = cola.popleft()
        estado_hash = frozenset(estado_actual.items())
        if estado_hash in visitados:
            continue
        visitados.add(estado_hash)

        if es_objetivo(estado_actual):
            return plan

        sucesores = []
        for c in celdas:
            nuevo_estado = aplicar_accion_estado(estado_actual, c, vecinos)
            nuevo_plan = plan + [f'pulsar {c}']
            sucesores.append((nuevo_estado, nuevo_plan))

        for est, p in sucesores:
            if es_objetivo(est):
                return p

        sucesores.sort(key=lambda x: similitud(x[0]), reverse=True)
        cola.extend(sucesores[:n])

    return None


reader = PDDLReader()
problema = reader.parse_problem('Dominio.pddl', 'Problema.pddl')

plan = busqueda_heuristica_pddl(problema, n=5)
if plan:
    print(f"Plan encontrado en {len(plan)} pasos:")
    for i, paso in enumerate(plan, 1):
        print(f"Paso {i}: {paso}")
else:
    print("No se encontró plan con esta búsqueda heurística.")

{'o__0_2_': ['o__1_2_', 'o__0_1_'], 'o__0_0_': ['o__1_0_', 'o__0_1_'], 'o__1_1_': ['o__0_1_', 'o__2_1_', 'o__1_0_', 'o__1_2_'], 'o__1_2_': ['o__0_2_', 'o__2_2_', 'o__1_1_'], 'o__2_2_': ['o__1_2_', 'o__2_1_'], 'o__1_0_': ['o__0_0_', 'o__2_0_', 'o__1_1_'], 'o__2_0_': ['o__1_0_', 'o__2_1_'], 'o__2_1_': ['o__1_1_', 'o__2_0_', 'o__2_2_'], 'o__0_1_': ['o__1_1_', 'o__0_0_', 'o__0_2_']}
{'o__0_2_': False, 'o__0_0_': False, 'o__1_1_': False, 'o__1_2_': False, 'o__2_2_': False, 'o__1_0_': False, 'o__2_0_': False, 'o__2_1_': False, 'o__0_1_': False}
Plan encontrado en 5 pasos:
Paso 1: pulsar o__1_1_
Paso 2: pulsar o__0_2_
Paso 3: pulsar o__0_0_
Paso 4: pulsar o__2_2_
Paso 5: pulsar o__2_0_
